In [8]:
import numpy as np
import sys
import json
import os
import utils
from scipy.special import softmax as softmax

np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=1000)


#inputs
coarse=0           #coarse = 0 ; non-coarse = 1
dth=12             #threshold for data [Available:5,12,15]
M=2                #memory states [Available:1,2,3,4]
A=4                #action size 4 -> [up,left,right,up]
replica=0          #if you wish to do multiple replicas

max_obs = 10       # distinct observations for rewards
O = 2              # distinct observations for actions [0, 1, 2, .., O-1]

symmetry=1
replica=0          #replica -- for non-coarse policies

#Dimension of grid: Lx by Ly
Lx = 92            
Ly = 131

#Location of source (Lx0,Ly0)
Lx0 = 45.5
Ly0 = 91

if coarse == 1:    #change size and center for coarse set-up
    Lx=91
    Lx0=45

    
lr=0.01            #learning rate
find_range=1.1     #radius from source location to be considered as found target
gamma=0.99975      #discount rate
V=100              #wind velocity for model plume


#combined action space = M * A, lM0, lM1, ... rM0, rM1, ..., uM0, uM1, ..., dM0, dM1, ...
a_size = A * M
L = Lx*Ly

# cost move!
cost_move = 1-gamma
reward_find=0.0

# probability of track
beta = 1
sigma = 4

#Maxtime in optimization
Ntot=1000000
#Printing time interval
Nprint=10

# ++++++++++++++++++
# CHANGE OF SCALES
scale = 1
L = L*scale
sigma = sigma*scale
gamma = 1 - (1-gamma)/scale
beta = beta/scale

print('Lx :{}, Ly :{}'.format(Lx, Ly))
print('Lx0:{}, Ly0:{}'.format(Lx0, Ly0))
print('M:{}, th:{}, replica:{}'.format(M, dth,replica))

if symmetry == 0:
    data=np.loadtxt('data/exp_plume_threshold{}.dat'.format(dth))
if symmetry == 1:
    data=np.loadtxt('data/exp_plume_symmetric_threshold{}.dat'.format(dth))
if coarse == 1:
    data=np.loadtxt('data/exp_plume_symmetric_coarse_threshold{}.dat'.format(dth))

#INITIALIZATIONS
eta = np.zeros(L*M)
PObs_lim, RR, PObs = utils.create_PObs_RR(Lx, Ly, Lx0, Ly0, find_range, cost_move, 
                                    reward_find, M, beta, max_obs, O, A, V, data=data)
PObs_lim = np.abs(PObs_lim)

# Initial density is only at y=0, proportional to signal probability
rho0 = np.zeros(M*Lx*Ly)
#rho0[:Lx*Ly0//2] = (1-PObs_lim[0,:Lx*Ly0//2])/np.sum((1-PObs_lim[0,:Lx*Ly0//2]))
rho0[:Lx] = (1-PObs_lim[0,:Lx])/np.sum((1-PObs_lim[0,:Lx]))
    
# Tolerance default
tol_eta = 0.000001
tol_Q = 0.000001
lr_th = 0.001
tol_conv = 0.0000000001
print('tol_conv:{}, tolQ:{}, toleta:{}'.format(tol_conv,tol_Q,tol_eta)) 
    
name_folder = 'ExpPlume_A{}M{}b{}g{}FR{}LR{}sym{}th{}rep{}LX{}LY{}co{}'.format(A,M,beta,gamma,find_range,lr_th,symmetry,dth,replica,Lx,Ly,coarse)
os.makedirs(name_folder, exist_ok=True)
#os.system("cp {} {}".format('./'+sys.argv[1], './'+name_folder))
f = open(name_folder+"/values.dat", "a")

Lx :92, Ly :131
Lx0:45.5, Ly0:91
M:2, th:12, replica:0
tol_conv:1e-10, tolQ:1e-06, toleta:1e-06


In [9]:
# Policy Initialization with bias
restart = False
unbias = False    
if (restart):
    folder_restart = name_folder
    th = np.loadtxt(folder_restart + '/file_theta.out')
    th = th.reshape(O, M, a_size)
    Q = np.loadtxt(folder_restart + '/file_Q.out')
    eta = np.loadtxt(folder_restart + '/file_eta.out')
else:
    th = (np.random.rand(O, M, a_size)-0.5)*0.5 
    th[1:,:,2::A] += 0.5
    if (unbias):
        th[1:,:,2::A] -= 0.5
    eta = np.ones(eta.shape)/(L*M)/(1-gamma)
    Q = utils.create_random_Q0(Lx, Ly, Lx0, Ly0, gamma, a_size, M, cost_move, reward_find) 
       
pi = softmax(th, axis=2)

value = 0
oldvalue = value

In [ ]:
#OPTIMIZATION ALGORITHM: NPG
for t in range(Ntot):
    pi = softmax(th, axis=2)
    
    # Iterative solutions of linear system
    eta = utils.iterative_solve_eta(pi, PObs_lim, gamma, rho0, eta, tol_eta, Lx, Ly, Lx0, Ly0, find_range)
    Q = utils.iterative_solve_Q(pi, PObs_lim, gamma, RR, Q, tol_Q, Lx, Ly, Lx0, Ly0, find_range, cost_move)

    # Gradient calculation
    grad = utils.find_grad(pi, Q, eta, L, PObs_lim)
    grad -= np.max(grad, axis=2, keepdims=True)

    # Reset value for new iteration afterwards

    th += grad * lr_th / np.max(np.abs(grad)) # (t / Ntot + 0.5) #rescaled gradient
    th -= np.max(th, axis=2, keepdims=True)
    th = np.clip(th, -20, 0)

    # Print and check convergence
    if (t % Nprint == 0):
        value =  utils.get_value(Q, pi, PObs_lim, L, rho0)
        #print values in a file
        f.write('current value: {} @ time:{} \n'.format(value, t))
        f.flush()

        # check convergence
        if (abs((value-oldvalue)/value)<tol_conv):
            f.write('converged at T={}'.format(t))
            break
        oldvalue = value

    #if (t % (Nprint*10) == 0): #save every Nprint*10 in different files
    #    np.savetxt(name_folder + '/file_theta{}.out'.format(t), th.reshape(-1))
    #    np.savetxt(name_folder + '/file_Q{}.out'.format(t), Q.reshape(-1))
    #    np.savetxt(name_folder + '/file_eta{}.out'.format(t), eta.reshape(-1))
    if (t % (Nprint*10) == 0)   :#save every Nprint*10 overwriting previous files
        np.savetxt(name_folder + '/file_theta.out', th.reshape(-1))
        np.savetxt(name_folder + '/file_Q.out', Q.reshape(-1))
        np.savetxt(name_folder + '/file_eta.out', eta.reshape(-1))

# final print of converged policy (or if not converged, until Ntot)
np.savetxt(name_folder + '/file_theta.out', th.reshape(-1))
np.savetxt(name_folder + '/file_Q.out', Q.reshape(-1))
np.savetxt(name_folder + '/file_eta.out', eta.reshape(-1))
if t<Ntot:
    print('converged at T={} with value={}'.format(t,value))
else:
    print('did not converged for {} runs and tolerance {}'.format(Ntot,))
    print('current value: {} @ time:{} \n'.format(value, t))

In [ ]:
#policy
print('policy p(a,m*|m,y)')
print('for y=o')
print(pi.reshape(O,M,M,A)[0])
print('for y=1')
print(pi.reshape(O,M,M,A)[1])

In [ ]:
#Visualize after optimization
#Sample Trajectory

trj, ret, _ = utils.single_traj_obs(softmax(th, axis=2), Lx, Ly, Lx0, Ly0, find_range, gamma, PObs_lim, rho0, A)
scatter_x = trj[1:,1]
scatter_y = trj[1:,2]
group = trj[1:,3]
cdict = {2: 'orange', 0: 'mediumseagreen', 1: 'crimson', 3:'blue'}

fig, ax = plt.subplots()

ix = np.where(trj[1:,4] > 0)
ax.scatter(scatter_x[ix], scatter_y[ix], c = 'black', s = 80)
ix = np.where(group == 0.0)
ax.scatter(scatter_x[ix], scatter_y[ix], c = cdict[0], label = 'Memory 0', s = 8)
ix = np.where(group == 1.0)
ax.scatter(scatter_x[ix], scatter_y[ix], c = cdict[1], label = 'Memory 1', s = 8)
ix = np.where(group == 2.0)
ax.scatter(scatter_x[ix], scatter_y[ix], c = cdict[2], label = 'Memory 2', s = 8)
ix = np.where(group == 3.0)
ax.scatter(scatter_x[ix], scatter_y[ix], c = cdict[3], label = 'Memory 3', s = 8)
#ax.legend()
ax.imshow((1-PObs_lim[0,:]-5).reshape(M,Ly,Lx)[0,:],cmap='Greys')
fig.set_size_inches(10, 6)
crange=plt.Circle((Lx0,Ly0),find_range,fill=False)
ax.set_aspect(1)

ax.add_artist(crange)
plt.title('{} threshold: {} memory states'.format(dth, M))
plt.xlim((-1,92))
plt.ylim((-1,133))
plt.xlabel('x-position',fontsize=16)
plt.ylabel('y-position',fontsize=16)
plt.show()

In [ ]:
#TIME DISTRIBUTIONS of search time 
#timedistributions

Nep = 1000    #number of trajectories to reproduce
av_ret = 0.
T = []
maxT = 10000  #maximum search time, not found before maxT would be counted as "failed search"

for i in range(Nep):
    _, ret, tau = utils.single_traj_obs(softmax(th, axis=2), Lx, Ly, Lx0, Ly0, find_range, maxT, PObs_lim, rho0, A)
    av_ret += ret
    T.append(tau)
print('Success Rate: {}%'.format(100*av_ret / Nep))
